# Playground

# Numba

In [ ]:
from src.Sudoku import Sudoku
import numpy as np


grid1 = """000801000
    000000043
    500000000
    000070800
    000000100
    020030000
    600000075
    003400000
    000200600"""

sudoku = Sudoku(grid1)


In [ ]:
from numba import jit, prange

def calculate_candidates(sudoku):

    candidates = np.zeros([9, 9]) 

    for y in range(9):
        for x in range(9):
            if sudoku.get_cell(x, y) != 0:
                candidates[y][x] = 0
                continue

            row = sudoku.get_row(y)
            col = sudoku.get_col(x)
            sq_index = x // 3 + 3 * (y // 3)
            sq = sudoku.get_square(sq_index)
            invalid_numbers = np.concatenate((row, col, sq.flatten()))
            valid_numbers = np.setdiff1d(np.arange(1, 10), invalid_numbers)
            candidates[y][x] = len(valid_numbers)

    return candidates


@jit(nopython=True)
def custom_setdiff1d(ar1, ar2):
    # Create an empty list to hold the result
    result = []
    
    # Convert ar2 to a set for faster lookup
    ar2_set = set(ar2)
    
    # Add elements to result that are in ar1 but not in ar2
    for item in ar1:
        if item not in ar2_set:
            result.append(item)
    
    return np.array(result)

@jit(nopython=True, parallel=True)
def calculate_candidates_2(grid: np.array):

    candidates = np.zeros((9, 9, 9), dtype=np.bool_)

    for y in prange(9):
        for x in prange(9):
            if grid[y, x] != 0:
                continue

            row = grid[y, :]
            col = grid[:, x]
            y0 = (y // 3) * 3
            x0 = (x // 3) * 3
            sq = grid[y0 : y0 + 3, x0 : x0 + 3]
            invalid_numbers = np.concatenate((row, col, sq.flatten()))
            valid_numbers = custom_setdiff1d(np.arange(1, 10), invalid_numbers)
            candidates[y, x, valid_numbers - 1] = 1

    return candidates


In [ ]:
import time

start_time = time.time()

for _ in range(1000000):
    c = calculate_candidates_2(sudoku.grid)

end_time = time.time()

print(f"Time: {end_time - start_time}")


Time: 54.25662636756897


In [ ]:
def decode_candidates(c):
    candidates = []
    for row in c:
        new_row = []
        for elem in row:
            new_row.append([i + 1 for i, e in enumerate(elem) if e])
        candidates.append(new_row)
    
    return candidates

            

In [ ]:
print(sudoku)

- - - 8 - 1 - - -
- - - - - - - 4 3
5 - - - - - - - -
- - - - 7 - 8 - -
- - - - - - 1 - -
- 2 - - 3 - - - -
6 - - - - - - 7 5
- - 3 4 - - - - -
- - - 2 - - 6 - -



In [ ]:
decode_candidates(c)

[[[2, 3, 4, 7, 9],
  [3, 4, 6, 7, 9],
  [2, 4, 6, 7, 9],
  [],
  [2, 4, 5, 6, 9],
  [],
  [2, 5, 7, 9],
  [2, 5, 6, 9],
  [2, 6, 7, 9]],
 [[1, 2, 7, 8, 9],
  [1, 6, 7, 8, 9],
  [1, 2, 6, 7, 8, 9],
  [5, 6, 7, 9],
  [2, 5, 6, 9],
  [2, 5, 6, 7, 9],
  [2, 5, 7, 9],
  [],
  []],
 [[],
  [1, 3, 4, 6, 7, 8, 9],
  [1, 2, 4, 6, 7, 8, 9],
  [3, 6, 7, 9],
  [2, 4, 6, 9],
  [2, 3, 4, 6, 7, 9],
  [2, 7, 9],
  [1, 2, 6, 8, 9],
  [1, 2, 6, 7, 8, 9]],
 [[1, 3, 4, 9],
  [1, 3, 4, 5, 6, 9],
  [1, 4, 5, 6, 9],
  [1, 5, 6, 9],
  [],
  [2, 4, 5, 6, 9],
  [],
  [2, 3, 5, 6, 9],
  [2, 4, 6, 9]],
 [[3, 4, 7, 8, 9],
  [3, 4, 5, 6, 7, 8, 9],
  [4, 5, 6, 7, 8, 9],
  [5, 6, 9],
  [2, 4, 5, 6, 8, 9],
  [2, 4, 5, 6, 8, 9],
  [],
  [2, 3, 5, 6, 9],
  [2, 4, 6, 7, 9]],
 [[1, 4, 7, 8, 9],
  [],
  [1, 4, 5, 6, 7, 8, 9],
  [1, 5, 6, 9],
  [],
  [4, 5, 6, 8, 9],
  [4, 5, 7, 9],
  [5, 6, 9],
  [4, 6, 7, 9]],
 [[],
  [1, 4, 8, 9],
  [1, 2, 4, 8, 9],
  [1, 3, 9],
  [1, 8, 9],
  [3, 8, 9],
  [2, 3, 4, 9],
  [],
  []],
 [[1

# The hardest grid

In [1]:
grid = [
  [9, 0, 0, 0, 8, 0, 0, 0, 5],
  [0, 0, 0, 0, 0, 0, 0, 0, 6],
  [0, 0, 0, 0, 0, 0, 0, 0, 7],
  [0, 8, 0, 0, 0, 5, 0, 0, 4],
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 4, 0, 2, 3, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 3, 0, 0, 0, 0, 0, 2],
  [2, 0, 0, 0, 0, 9, 0, 8, 3]
]

In [4]:
from src.Sudoku import Sudoku
from src.SudokuAnalyzer import SudokuAnalyzer
from src.SudokuScrambler import SudokuScrambler
from src.SudokuSolver import SudokuSolver

sudoku = Sudoku(grid)

In [5]:
grid = [[3, 6, 5, 0, 1, 0, 4, 0, 0],
 [0, 9, 0, 0, 0, 6, 0, 0, 0],
 [0, 0, 0, 3, 8, 0, 2, 0, 6],
 [0, 7, 0, 6, 2, 3, 0, 0, 8],
 [1, 0, 0, 7, 0, 4, 3, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 4, 2],
 [0, 0, 3, 2, 0, 0, 0, 5, 1],
 [9, 0, 0, 0, 0, 0, 0, 0, 3],
 [0, 0, 1, 0, 0, 0, 8, 0, 0]]

In [6]:
sudoku = Sudoku(grid)
sudoku.get_hash()

'365010400090006000000380206070623008100704300000001042003200051900000003001000800'

In [50]:
non_zeros = []
for y in range(9):
    for x in range(9):
        if grid[y][x]:
            non_zeros.append((x, y))


In [51]:
sudoku = Sudoku(grid)
analyzer = SudokuAnalyzer(sudoku)
analyzer.get_sudoku_description()

(151, 31, 3.06)

In [52]:
from random import shuffle

shuffle(non_zeros)

for x, y in non_zeros:
    tmp, grid[y][x] = grid[y][x], 0
    sudoku = Sudoku(grid)
    solver = SudokuSolver(sudoku)
    result = solver.solve()
    if result is None:
    #    print(f"Cannot remove {x} {y} {tmp}")
    
    grid[y][x] = tmp
    
    


In [2]:
sudoku = Sudoku(grid)
analyzer = SudokuAnalyzer(sudoku)
analyzer.get_sudoku_description()

NameError: name 'Sudoku' is not defined

# Recursive removal of digits


In [29]:
from src.Sudoku import Sudoku
from src.SudokuSolver import SudokuSolver
from src.SudokuAnalyzer import SudokuAnalyzer
from copy import deepcopy


def remove_digits(grids: list, desired_deepnees: int, limit = 1000, filename = "grids.txt"):
    results = []
    found_grids = 0
    valid_grids = set()

    with open(filename, 'r') as f:
        valid_grids = set([line.strip() for line in f])

    print(len(valid_grids))


    def recursive_removal(sudoku: Sudoku, first_solution: Sudoku, list_of_non_zeros: list, n: int = 0):
        nonlocal limit
        nonlocal results
        nonlocal found_grids

        if found_grids == limit:
            return None

        if sudoku.get_hash() in valid_grids:
            return None
        
        solver = SudokuSolver(sudoku)
        if solver.solve_recursive(ignore_solution=first_solution) is not None:
            sudoku.reset_sudoku()
            return None

        valid_grids.add(sudoku.get_hash())

        if n == desired_deepnees:
            found_grids += 1
            with open(filename, 'a') as f:
                f.write(f"{sudoku.get_hash()}\n")
            results.append(deepcopy(sudoku.grid))
            return None

        for i, coords in enumerate(list_of_non_zeros):
            tmp = sudoku.get_cell(coords[0], coords[1])
            sudoku.initial_grid[coords[1], coords[0]] = 0
            sudoku.set_cell(coords[0], coords[1], 0)

            recursive_removal(
                sudoku, first_solution, list_of_non_zeros[:i] + list_of_non_zeros[i + 1 :], n + 1
            )
            
            sudoku.set_cell(coords[0], coords[1], tmp)
            sudoku.initial_grid[coords[1], coords[0]] = tmp
        
        return None
    
    for g in grids:
        sudoku = Sudoku(g)
        found_grids = 0
     
        non_zeros = []
        for y in range(9):
            for x in range(9):
                if sudoku.grid[y][x]:
                    non_zeros.append((x, y))

        solver = SudokuSolver(sudoku)
        solution = deepcopy(solver.solve_recursive())
        sudoku.reset_sudoku()

        recursive_removal(sudoku, solution, non_zeros)

    return results

def hash_to_grid(hash: str):
    grid = ""
    for i in range(0, 81, 9):
        grid += hash[i : i + 9] + "\n"

    return grid

def remove_duplicated_lines_from_file(filename):
    with open(filename, "r+") as f:
        hashes = [line.strip() for line in f]
        hashes = set(hashes)
        f.seek(0)
        for h in hashes:
            f.write(f"{h}\n")
        f.truncate()

In [27]:
remove_duplicated_lines_from_file(f"grids/grids_{26}.txt")


In [31]:
root_grids_limit = 2000
sub_grids_limit = 100

for i in range(23, 19, -1):

    with open(f"grids/grids_{i}.txt") as f:
        hashes = [line.strip() for line in f]

    grids = [hash_to_grid(h) for h in hashes]
    lenght = len(grids)


    new_grids = remove_digits(grids[:root_grids_limit], 1, sub_grids_limit, f"grids/grids_{i - 1}.txt")
    remove_duplicated_lines_from_file(f"grids/grids_{i - 1}.txt")

    with open("debug.txt", 'a') as f:
        f.write(f"{i} done\n")

264
19
1


FileNotFoundError: [Errno 2] No such file or directory: 'grids/grids_19.txt'

In [9]:
grid = hash_to_grid("100020000600000800000000106004500320090600000000030000057200090009080000000400071")
sudoku = Sudoku(grid)
analyzer = SudokuAnalyzer(sudoku)
analyzer.get_sudoku_description()

(240, 22, 3.14)

In [35]:
my_gridooo = deepcopy(new_grids)

In [57]:
from random import shuffle
shuffle(my_gridooo)

new_grids = []
for g in my_gridooo:
    grids = produce_subgrids(g, 3, 10, "grids_22.txt")
    new_grids.extend(grids)

len(new_grids)

KeyboardInterrupt: 

In [46]:
def save_grids(grids, file):
    with open(file, 'a') as f:
        for g in grids:
            sudoku = Sudoku(g)
            analyzer = SudokuAnalyzer(sudoku)
            properties = analyzer.get_sudoku_description()
            f.write(f"{properties[0]};{properties[1]};{properties[2]}\n")
            f.write(f"{sudoku.get_hash()}\n")

In [48]:
save_grids(my_gridooo, "grids_25.txt")

In [45]:
len(my_gridooo)

4344

In [47]:
g = my_gridooo[0]
sudoku = Sudoku(g)
analyzer = SudokuAnalyzer(sudoku)
analyzer.get_sudoku_description()

(217, 25, 3.06)

In [ ]:
from copy import deepcopy

grid = [
    [3, 6, 5, 0, 1, 0, 4, 0, 0],
    [0, 9, 0, 0, 0, 6, 0, 0, 0],
    [0, 0, 0, 3, 8, 0, 2, 0, 6],
    [0, 7, 0, 6, 2, 3, 0, 0, 8],
    [1, 0, 0, 7, 0, 4, 3, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 4, 2],
    [0, 0, 3, 2, 0, 0, 0, 5, 1],
    [9, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 1, 0, 0, 0, 8, 0, 0],
]
sudoku = Sudoku(grid)
solver = SudokuSolver(sudoku)
first_solution = deepcopy(solver.solve())

results = []
sub_grids_to_check = list(new_grid[1])
for x, i in enumerate(new_grid[1]):
    grid = ""
    for j in range(0, 81, 9):
        grid += i[j : j + 9]
        grid += "\n"

    sudoku = Sudoku(grid)
    solver = SudokuSolver(sudoku)
    # sol = solver.solve_recursive(ignore_solution=first_solution)
    sol = solver.solve()
    results.append((sudoku, sol is None))
    # print(sudoku)


In [10]:
results = [i[0] for i in results if i[1]]

for i in results:
    print(i)

- 6 - - - - 4 - - 
- - - - - 6 - - - 
- - - 3 8 - - - 6 
- 7 - 6 2 3 - - 8 
1 - - 7 - 4 3 - - 
- - - - - 1 - 4 2 
- - 3 2 - - - 5 1 
9 - - - - - - - 3 
- - 1 - - - 8 - - 

- 6 - - - - 4 - - 
- - - - - 6 - - - 
- - - 3 8 - - - - 
- 7 - 6 2 3 - - 8 
1 - - 7 - 4 3 - - 
- - - - - 1 - 4 2 
- - 3 2 - - - 5 1 
9 - - - - - - - 3 
- - 1 - - - 8 - - 



In [5]:
from src.Sudoku import Sudoku
from src.SudokuSolver import SudokuSolver

grid = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 0, 0, 9, 0, 0],
    [0, 0, 9, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 7, 0, 6, 9, 0],
    [0, 0, 0, 9, 0, 0, 0, 7, 8],
    [0, 0, 0, 6, 0, 0, 2, 0, 0],
    [8, 0, 2, 0, 0, 0, 5, 6, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 5, 0, 0, 0, 0, 0, 0],
]
sudoku = Sudoku(grid)
solver = SudokuSolver(sudoku)
sol = solver.solve()
print(sol)

None


In [18]:
grid = """568912437
030006080
010380006
475623008
100704360
006801742
003200051
090000003
601030804"""

sudoku = Sudoku(grid)
solver = SudokuSolver(sudoku)
solver.grid_of_candidates[6][0]

sudoku.set_cell(0, 1, 2)
solver.update_grid_of_candidates(0, 1)
for row in solver.grid_of_candidates:
    for v in row:
        if not v:
            print("0 ", end="")
        else:
            print("".join([str(i) for i in v]), end=" ")
    print()
print("######\n" *3)

solver.apply_advance_rules()
for row in solver.grid_of_candidates:
    for v in row:
        if not v:
            print("0 ", end="")
        else:
            print("".join([str(i) for i in v]), end=" ")
    print()

print(solver.grid_of_candidates[6][0])

-1 -1 -1 -1 -1 -1 -1 -1 -1 
-1 -1 479 45 457 -1 159 -1 59 
79 -1 479 -1 -1 57 259 29 -1 
-1 -1 -1 -1 -1 -1 19 19 -1 
-1 28 29 -1 59 -1 -1 -1 59 
39 0 -1 -1 59 -1 -1 -1 -1 
78 48 -1 -1 4679 789 69 -1 -1 
78 -1 247 145 4567 578 26 27 -1 
-1 25 -1 5 -1 579 -1 279 -1 
######
######
######

-1 -1 -1 -1 -1 -1 -1 -1 -1 
-1 -1 7 4 7 -1 1 -1 9 
9 -1 4 -1 -1 7 5 2 -1 
-1 -1 -1 -1 -1 -1 9 1 -1 
-1 8 9 -1 5 -1 -1 -1 5 
3 0 -1 -1 9 -1 -1 -1 -1 
7 4 -1 -1 6 8 9 -1 -1 
8 -1 2 1 4 5 6 7 -1 
-1 5 -1 5 -1 9 -1 2 -1 
[7]


In [44]:
grid = [
    [3, 6, 5, 0, 1, 0, 4, 0, 0],
    [0, 9, 0, 0, 0, 6, 0, 0, 0],
    [0, 0, 0, 3, 8, 0, 2, 0, 6],
    [0, 7, 0, 6, 2, 3, 0, 0, 8],
    [1, 0, 0, 7, 0, 4, 3, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 4, 2],
    [0, 0, 3, 2, 0, 0, 0, 5, 1],
    [9, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 1, 0, 0, 0, 8, 0, 0],
]
sudoku = Sudoku(grid)
solver = SudokuSolver(sudoku)


In [45]:
result = deepcopy(solver.solve_recursive())
sudoku.reset_sudoku()

In [10]:
v = []
if v.empty()

False

In [47]:
print(sudoku)

3 6 5 - 1 - 4 - - 
- 9 - - - 6 - - - 
- - - 3 8 - 2 - 6 
- 7 - 6 2 3 - - 8 
1 - - 7 - 4 3 - - 
- - - - - 1 - 4 2 
- - 3 2 - - - 5 1 
9 - - - - - - - 3 
- - 1 - - - 8 - - 

